# Exploratory Data Analysis

In [ ]:
import pandas as pd

from income_predict_d100_d400 import cleaning
from income_predict_d100_d400.robust_paths import DATA_DIR
from income_predict_d100_d400 import eda

In [ ]:
parquet_path = DATA_DIR / "census_income.parquet"

df_raw = pd.read_parquet(parquet_path)
print(df_raw.head())


In [ ]:
df_raw.info()
df_raw.describe()

In [ ]:
#1. Describe your data.
eda.get_data_description(df_raw)

In [ ]:
#2. What is the distribution of the target variable?
eda.get_target_distribution(df_raw, 'income')


In [ ]:
#3. Do we face outliers and missing values?
eda.get_outliers_summary(df_raw)

In [ ]:
# Cleaning pipeline
df_clean = cleaning.full_clean(df_raw)
df_clean.columns.tolist()

In [ ]:
#4. How do specific features correlate with the target variable?
eda.get_feature_correlations(df_clean, 'high_income')